In [ ]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from botocore import UNSIGNED
from botocore.config import Config
from matplotlib.gridspec import GridSpec

# --- 1. SETUP STYLING CONSTANTS ---
LS = ['-', '--', '-.', ':'] 
CB_PALETTE = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# --- 2. DATA DOWNLOAD ---
s3 = boto3.client('s3', region_name='us-west-2', config=Config(signature_version=UNSIGNED))
bucket = 'rover-mobility-in-a-lunar-analogue-dataset'
key = 'Trial_Data/5.0_120.0_Right_0.0/Trial_1/Processed_Data.csv'

print("Downloading data from S3...")
obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(obj['Body'])
print("Data loaded successfully.")

# --- 3. UPDATED PLOTTING FUNCTION ---
def make_overview_2x4_slides(df):
    """
    Overview figure optimized for Google Slides (16:9 aspect ratio).
    Uses raw column names from the provided dataframe:
    - time_s, pos_x, quat_x, enc1, current1, etc.
    """
    
    # 1. Extract Time
    t = df['time_s'].to_numpy()

    # 2. Extract Position & Velocity calculations
    vx = np.gradient(df['pos_x'].to_numpy(), t)
    vy = np.gradient(df['pos_y'].to_numpy(), t)
    speed = np.hypot(vx, vy)
    
    # Determine if we need to flip the Y-axis for visualization
    moving = speed > max(np.percentile(speed, 90) * 0.2, 0.005)
    y_plot = df['pos_y'].copy()
    if np.nanmean(df['pos_y'][moving]) < 0:
        y_plot = -y_plot

    # 3. Prepare Wheel Data
    thetas = [df[f'enc{i}'].to_numpy() * 2*np.pi for i in range(1,5)]
    omegas = [df[f'vel{i}'].to_numpy() * 2*np.pi for i in range(1,5)]
    
    # Mapping 'current{i}'
    currents = [df[f'current{i}'].to_numpy() for i in range(1,5)]

    # 4. Plotting Setup
    plt.rcParams.update({'font.size': 11}) 
    fig = plt.figure(figsize=(13, 6), dpi=150) 
    gs = GridSpec(2, 4, figure=fig, height_ratios=[1, 1])

    # --- Row 1 ---

    # Row 1, col 1: Rover Position (pos_x, pos_y, pos_z)
    ax = fig.add_subplot(gs[0, 0])
    ax.plot(t, df['pos_x'], label='px', linestyle=LS[0], linewidth=1)
    ax.plot(t, y_plot,      label='py', linestyle=LS[1], linewidth=1)
    ax.plot(t, df['pos_z'], label='pz', linestyle=LS[2], linewidth=1)
    ax.set_ylabel('Position (m)')
    ax.set_title('Rover Position')
    ax.legend(fontsize=8)
    ax.tick_params(labelbottom=False)

    # Row 1, col 2: Rover Orientation (quat_x, quat_y, quat_z, quat_w)
    ax = fig.add_subplot(gs[0, 1])
    ax.plot(t, df['quat_x'], label='qx', linestyle=LS[0], linewidth=1)
    ax.plot(t, df['quat_y'], label='qy', linestyle=LS[1], linewidth=1)
    ax.plot(t, df['quat_z'], label='qz', linestyle=LS[2], linewidth=1)
    ax.plot(t, df['quat_w'], label='qw', linestyle=LS[3], linewidth=1)
    ax.set_ylabel('Quaternion')
    ax.set_title('Rover Orientation')
    ax.legend(fontsize=8, ncol=2)
    ax.tick_params(labelbottom=False)

    # Row 1, col 3: Linear Acceleration 
    ax = fig.add_subplot(gs[0, 2])
    try:
        ax.plot(t, df['lin_acc_x'], label='ax', linestyle=LS[0], linewidth=1)
        ax.plot(t, df['lin_acc_y'], label='ay', linestyle=LS[1], linewidth=1)
        ax.plot(t, df['lin_acc_z'], label='az', linestyle=LS[2], linewidth=1)
    except KeyError:
        print("Warning: Could not find 'lin_accel_x'. Plotting empty graph for Accel.")
        
    ax.set_ylabel('Accel (m/s²)')
    ax.set_title('Rover Lin Accel')
    ax.legend(fontsize=8)
    ax.tick_params(labelbottom=False)

    # Row 1, col 4: Angular Velocity (ang_vel_x, ang_vel_y, ang_vel_z)
    ax = fig.add_subplot(gs[0, 3])
    ax.plot(t, df['ang_vel_x'], label='ωx', linestyle=LS[0], linewidth=1)
    ax.plot(t, df['ang_vel_y'], label='ωy', linestyle=LS[1], linewidth=1)
    ax.plot(t, df['ang_vel_z'], label='ωz', linestyle=LS[2], linewidth=1)
    ax.set_ylabel('Ang Vel (rad/s)')
    ax.set_title('Rover Ang Vel')
    ax.legend(fontsize=8)
    ax.tick_params(labelbottom=False)

    # --- Row 2 ---

    # Row 2, col 1: Wheel Angular Displacement
    ax = fig.add_subplot(gs[1, 0])
    for i, theta in enumerate(thetas, start=1):
        ax.plot(t, theta, label=f'W{i}', linestyle=LS[(i-1) % len(LS)], linewidth=1)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Angle (rad)')
    ax.set_title('Wheel Ang Disp')
    ax.legend(fontsize=8, ncol=2)

    # Row 2, col 2: Wheel Angular Velocity
    ax = fig.add_subplot(gs[1, 1])
    for i, omg in enumerate(omegas, start=1):
        ax.plot(t, omg, label=f'W{i}', linestyle=LS[(i-1) % len(LS)], linewidth=1)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Ang Vel (rad/s)')
    ax.set_title('Wheel Ang Vel')
    ax.legend(fontsize=8, ncol=2)

    # Row 2, col 3: Motor Currents
    ax = fig.add_subplot(gs[1, 2])
    for i, cur in enumerate(currents, start=1):
        ax.plot(t, cur, label=f'M{i}', linestyle=LS[(i-1) % len(LS)], linewidth=1)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Current (A)')
    ax.set_title('Motor Current')
    ax.legend(fontsize=8, ncol=2)

    # Row 2, col 4: Plan-view trajectory
    ax = fig.add_subplot(gs[1, 3])
    
    x_t = df['pos_x'].to_numpy()
    y_t = df['pos_y'].to_numpy()

    WIN_S = 27.0
    mask = t <= WIN_S
    if mask.sum() < 2:
        k = max(int(0.05 * len(t)), 2)
        mask = np.zeros_like(t, dtype=bool); mask[:k] = True

    # Rotation
    dx = x_t[mask][-1] - x_t[mask][0]
    dy = y_t[mask][-1] - y_t[mask][0]
    theta = np.arctan2(dy, dx)

    c, s = np.cos(-theta), np.sin(-theta)
    x_rot = c * x_t - s * y_t
    y_rot = s * x_t + c * y_t

    vx_early = np.gradient(x_rot[mask], t[mask])
    if np.nanmean(vx_early) < 0:
        x_rot = -x_rot
        y_rot = -y_rot

    ax.plot(y_rot, x_rot, color=CB_PALETTE[0], linewidth=1.5)
    ax.scatter(y_rot[0], x_rot[0], color='green', s=30, label='Start', zorder=5)
    ax.scatter(y_rot[-1], x_rot[-1], color='red', s=30, label='End', zorder=5)

    ax.set_xlabel('Lateral Y (m)')
    ax.set_ylabel('Forward X (m)')
    ax.set_title('Trajectory')
    ax.set_aspect('equal', 'box')
    ax.grid(True, linestyle=':', linewidth=0.5)
    ax.invert_xaxis()

    print(f"[note] Plan view rotated by {-np.degrees(theta):.2f} deg.")
    ax.legend(fontsize=8)
    plt.tight_layout()
    plt.show()

# --- 4. EXECUTE ---
make_overview_2x4_slides(df)